In [1]:
import mlflow

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716836631182, experiment_id='0', last_update_time=1716836631182, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/27 19:11:58 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1/e300c19b4f46442a8003e483632affaa/artifacts'


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-camp/notebook/03-experiment-tracking/running-mlflow-exp/artifacts_local/1', creation_time=1716837118065, experiment_id='1', last_update_time=1716837118065, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716836631182, experiment_id='0', last_update_time=1716836631182, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from mlflow.tracking import MlflowClient

In [9]:
client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.search_registered_models()

[]

In [14]:
run_id = client.search_experiments(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

TypeError: MlflowClient.search_experiments() got an unexpected keyword argument 'experiment_id'